In [91]:
import pandas as pd
import torchvision
import torch
from PIL import Image
import numpy as np

In [92]:
data_csv = pd.read_csv('train.csv')

In [95]:
train_mask = np.random.rand(len(data_csv)) < 0.9

data_csv_train = data_csv[train_mask]
data_csv_eval = data_csv[~train_mask]

6616

In [76]:
batch_size = 8
lr = 1e-4
num_epochs = 2

In [53]:
data_transforms = torchvision.transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [102]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, transform, data):
        self.files = ['data/train/' + x + '.jpg' for x in data['image_name']]
        self.labels = [0 if x == 'benign' else 1 for x in data['benign_malignant']]
        self.transform = transform
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        return (self.transform(Image.open(self.files[idx])), self.labels[idx])
    
    def get_untransformed(self, idx):
        return Image.open(self.files[idx])


dataset = MyDataset(data_transforms, data_csv_train)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

dataset_eval = MyDataset(data_transforms, data_csv_eval)
dataloader_eval = torch.utils.data.DataLoader(dataset_eval, batch_size=128, num_workers=4)

In [72]:
model = torchvision.models.resnet18(pretrained=True)
# First go for feature extraction mode
for param in model.parameters():
    param.requires_grad = False

# Reinitialize the last layer of the model
model.fc = torch.nn.Linear(512, 2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [87]:
# The optimizer only needs to see the layers which have grads
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad:
        print(name)
        params_to_update.append(param)

optimizer = torch.optim.Adam(params_to_update, lr=lr)
criterion = torch.nn.CrossEntropyLoss()

fc.weight
fc.bias


In [103]:
def test():
    with torch.no_grad():
        total_loss = 0
        epochs = 0
        for images, labels in dataloader_eval:
            images = images.to(device)
            labels = labels.to(device)
            total_loss += criterion(model(images), labels)
            epochs += 1
        print("Eval loss: " + str(total_loss / epochs))

In [88]:
for epoch in range(num_epochs):
    print('Epoch ' + str(epoch) + ' of ' + str(num_epochs))
    model.train()
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        loss = criterion(model(images), labels)
        loss.backward()
        optimizer.step()
        print(loss)
    test()

Epoch 0 of 2
tensor(0.3567, grad_fn=<NllLossBackward>)
tensor(0.3149, grad_fn=<NllLossBackward>)
tensor(0.2992, grad_fn=<NllLossBackward>)
tensor(0.2718, grad_fn=<NllLossBackward>)
tensor(0.2449, grad_fn=<NllLossBackward>)
tensor(0.5384, grad_fn=<NllLossBackward>)
tensor(0.3363, grad_fn=<NllLossBackward>)
tensor(0.2118, grad_fn=<NllLossBackward>)
tensor(0.2043, grad_fn=<NllLossBackward>)
tensor(0.1785, grad_fn=<NllLossBackward>)
tensor(0.1668, grad_fn=<NllLossBackward>)
tensor(0.1572, grad_fn=<NllLossBackward>)
tensor(0.1422, grad_fn=<NllLossBackward>)
tensor(0.3681, grad_fn=<NllLossBackward>)
tensor(0.1308, grad_fn=<NllLossBackward>)
tensor(0.1185, grad_fn=<NllLossBackward>)
tensor(0.5120, grad_fn=<NllLossBackward>)
tensor(0.1238, grad_fn=<NllLossBackward>)
tensor(0.0934, grad_fn=<NllLossBackward>)
tensor(0.0962, grad_fn=<NllLossBackward>)
tensor(0.3640, grad_fn=<NllLossBackward>)
tensor(0.0877, grad_fn=<NllLossBackward>)
tensor(0.0927, grad_fn=<NllLossBackward>)
tensor(0.0825, grad_f

tensor(0.0130, grad_fn=<NllLossBackward>)
tensor(0.0155, grad_fn=<NllLossBackward>)
tensor(0.0119, grad_fn=<NllLossBackward>)
tensor(0.0137, grad_fn=<NllLossBackward>)
tensor(0.5131, grad_fn=<NllLossBackward>)
tensor(0.0115, grad_fn=<NllLossBackward>)
tensor(0.0124, grad_fn=<NllLossBackward>)
tensor(0.0123, grad_fn=<NllLossBackward>)
tensor(0.0131, grad_fn=<NllLossBackward>)
tensor(0.5931, grad_fn=<NllLossBackward>)
tensor(0.0140, grad_fn=<NllLossBackward>)
tensor(0.0134, grad_fn=<NllLossBackward>)
tensor(0.0144, grad_fn=<NllLossBackward>)
tensor(0.0153, grad_fn=<NllLossBackward>)
tensor(0.0210, grad_fn=<NllLossBackward>)
tensor(0.0152, grad_fn=<NllLossBackward>)
tensor(0.6075, grad_fn=<NllLossBackward>)
tensor(0.0128, grad_fn=<NllLossBackward>)
tensor(0.0140, grad_fn=<NllLossBackward>)
tensor(0.0155, grad_fn=<NllLossBackward>)
tensor(0.0152, grad_fn=<NllLossBackward>)
tensor(0.0201, grad_fn=<NllLossBackward>)
tensor(0.0191, grad_fn=<NllLossBackward>)
tensor(0.0130, grad_fn=<NllLossBac

tensor(0.0245, grad_fn=<NllLossBackward>)
tensor(0.0255, grad_fn=<NllLossBackward>)
tensor(0.0237, grad_fn=<NllLossBackward>)
tensor(0.0244, grad_fn=<NllLossBackward>)
tensor(0.0292, grad_fn=<NllLossBackward>)
tensor(0.0256, grad_fn=<NllLossBackward>)
tensor(0.5928, grad_fn=<NllLossBackward>)
tensor(0.0237, grad_fn=<NllLossBackward>)
tensor(0.0212, grad_fn=<NllLossBackward>)
tensor(0.0228, grad_fn=<NllLossBackward>)
tensor(0.0273, grad_fn=<NllLossBackward>)
tensor(0.0239, grad_fn=<NllLossBackward>)
tensor(0.5838, grad_fn=<NllLossBackward>)
tensor(0.0226, grad_fn=<NllLossBackward>)
tensor(0.0237, grad_fn=<NllLossBackward>)
tensor(0.0272, grad_fn=<NllLossBackward>)
tensor(0.0274, grad_fn=<NllLossBackward>)
tensor(0.0242, grad_fn=<NllLossBackward>)
tensor(0.0242, grad_fn=<NllLossBackward>)
tensor(0.0247, grad_fn=<NllLossBackward>)
tensor(0.0226, grad_fn=<NllLossBackward>)
tensor(0.6095, grad_fn=<NllLossBackward>)
tensor(0.0215, grad_fn=<NllLossBackward>)
tensor(0.0250, grad_fn=<NllLossBac

tensor(0.5224, grad_fn=<NllLossBackward>)
tensor(0.0215, grad_fn=<NllLossBackward>)
tensor(0.0211, grad_fn=<NllLossBackward>)
tensor(0.0247, grad_fn=<NllLossBackward>)
tensor(0.0213, grad_fn=<NllLossBackward>)
tensor(0.0252, grad_fn=<NllLossBackward>)
tensor(0.0242, grad_fn=<NllLossBackward>)
tensor(0.0299, grad_fn=<NllLossBackward>)
tensor(0.4739, grad_fn=<NllLossBackward>)
tensor(0.4870, grad_fn=<NllLossBackward>)
tensor(0.0236, grad_fn=<NllLossBackward>)
tensor(0.0233, grad_fn=<NllLossBackward>)
tensor(0.0222, grad_fn=<NllLossBackward>)
tensor(0.0254, grad_fn=<NllLossBackward>)
tensor(0.0280, grad_fn=<NllLossBackward>)
tensor(0.4164, grad_fn=<NllLossBackward>)
tensor(0.0245, grad_fn=<NllLossBackward>)
tensor(0.0244, grad_fn=<NllLossBackward>)
tensor(0.0317, grad_fn=<NllLossBackward>)
tensor(0.3553, grad_fn=<NllLossBackward>)
tensor(0.0270, grad_fn=<NllLossBackward>)
tensor(0.0264, grad_fn=<NllLossBackward>)
tensor(0.0223, grad_fn=<NllLossBackward>)
tensor(0.0271, grad_fn=<NllLossBac

tensor(0.0308, grad_fn=<NllLossBackward>)
tensor(0.0325, grad_fn=<NllLossBackward>)
tensor(0.4317, grad_fn=<NllLossBackward>)
tensor(0.0283, grad_fn=<NllLossBackward>)
tensor(0.0295, grad_fn=<NllLossBackward>)
tensor(0.0401, grad_fn=<NllLossBackward>)
tensor(0.0265, grad_fn=<NllLossBackward>)
tensor(0.0284, grad_fn=<NllLossBackward>)
tensor(0.4846, grad_fn=<NllLossBackward>)
tensor(0.0284, grad_fn=<NllLossBackward>)
tensor(0.0324, grad_fn=<NllLossBackward>)
tensor(0.0272, grad_fn=<NllLossBackward>)
tensor(0.0271, grad_fn=<NllLossBackward>)
tensor(0.0257, grad_fn=<NllLossBackward>)
tensor(0.0314, grad_fn=<NllLossBackward>)
tensor(0.3968, grad_fn=<NllLossBackward>)
tensor(0.0273, grad_fn=<NllLossBackward>)
tensor(0.0290, grad_fn=<NllLossBackward>)
tensor(0.0300, grad_fn=<NllLossBackward>)
tensor(0.0311, grad_fn=<NllLossBackward>)
tensor(0.0263, grad_fn=<NllLossBackward>)
tensor(0.0261, grad_fn=<NllLossBackward>)
tensor(0.0229, grad_fn=<NllLossBackward>)
tensor(0.0245, grad_fn=<NllLossBac

...

tensor(0.3461, grad_fn=<NllLossBackward>)
tensor(0.0195, grad_fn=<NllLossBackward>)
tensor(0.0146, grad_fn=<NllLossBackward>)
tensor(0.0195, grad_fn=<NllLossBackward>)
tensor(0.0195, grad_fn=<NllLossBackward>)
tensor(0.0364, grad_fn=<NllLossBackward>)
tensor(0.0173, grad_fn=<NllLossBackward>)
tensor(0.0176, grad_fn=<NllLossBackward>)
tensor(0.0216, grad_fn=<NllLossBackward>)
tensor(0.2765, grad_fn=<NllLossBackward>)
tensor(0.0164, grad_fn=<NllLossBackward>)
tensor(0.4202, grad_fn=<NllLossBackward>)
tensor(0.0165, grad_fn=<NllLossBackward>)
tensor(0.0359, grad_fn=<NllLossBackward>)
tensor(0.5295, grad_fn=<NllLossBackward>)
tensor(0.0199, grad_fn=<NllLossBackward>)
tensor(0.0196, grad_fn=<NllLossBackward>)
tensor(0.0178, grad_fn=<NllLossBackward>)
tensor(0.0382, grad_fn=<NllLossBackward>)
tensor(0.0165, grad_fn=<NllLossBackward>)
tensor(0.0158, grad_fn=<NllLossBackward>)
tensor(0.0160, grad_fn=<NllLossBackward>)
tensor(0.0152, grad_fn=<NllLossBackward>)
tensor(0.0159, grad_fn=<NllLossBac

tensor(0.0168, grad_fn=<NllLossBackward>)
tensor(0.0224, grad_fn=<NllLossBackward>)
tensor(0.0147, grad_fn=<NllLossBackward>)
tensor(0.0196, grad_fn=<NllLossBackward>)
tensor(0.1602, grad_fn=<NllLossBackward>)
tensor(0.0156, grad_fn=<NllLossBackward>)
tensor(0.0190, grad_fn=<NllLossBackward>)
tensor(0.0177, grad_fn=<NllLossBackward>)
tensor(0.0227, grad_fn=<NllLossBackward>)
tensor(0.0151, grad_fn=<NllLossBackward>)
tensor(0.0124, grad_fn=<NllLossBackward>)
tensor(0.0119, grad_fn=<NllLossBackward>)
tensor(0.0226, grad_fn=<NllLossBackward>)
tensor(0.0174, grad_fn=<NllLossBackward>)
tensor(0.0400, grad_fn=<NllLossBackward>)
tensor(0.0117, grad_fn=<NllLossBackward>)
tensor(0.4597, grad_fn=<NllLossBackward>)
tensor(0.5752, grad_fn=<NllLossBackward>)
tensor(0.0270, grad_fn=<NllLossBackward>)
tensor(0.0156, grad_fn=<NllLossBackward>)
tensor(0.0256, grad_fn=<NllLossBackward>)
tensor(0.0316, grad_fn=<NllLossBackward>)
tensor(0.0214, grad_fn=<NllLossBackward>)
tensor(0.0170, grad_fn=<NllLossBac

tensor(0.0400, grad_fn=<NllLossBackward>)
tensor(0.4021, grad_fn=<NllLossBackward>)
tensor(0.0193, grad_fn=<NllLossBackward>)
tensor(0.0238, grad_fn=<NllLossBackward>)
tensor(0.0181, grad_fn=<NllLossBackward>)
tensor(0.0189, grad_fn=<NllLossBackward>)
tensor(0.0173, grad_fn=<NllLossBackward>)
tensor(0.0182, grad_fn=<NllLossBackward>)
tensor(0.0153, grad_fn=<NllLossBackward>)
tensor(0.0276, grad_fn=<NllLossBackward>)
tensor(0.0361, grad_fn=<NllLossBackward>)
tensor(0.0200, grad_fn=<NllLossBackward>)
tensor(0.0276, grad_fn=<NllLossBackward>)
tensor(0.0143, grad_fn=<NllLossBackward>)
tensor(0.0237, grad_fn=<NllLossBackward>)
tensor(0.0161, grad_fn=<NllLossBackward>)
tensor(0.5533, grad_fn=<NllLossBackward>)
tensor(0.0170, grad_fn=<NllLossBackward>)
tensor(0.0133, grad_fn=<NllLossBackward>)
tensor(0.0186, grad_fn=<NllLossBackward>)
tensor(0.0145, grad_fn=<NllLossBackward>)
tensor(0.0141, grad_fn=<NllLossBackward>)
tensor(0.0163, grad_fn=<NllLossBackward>)
tensor(0.4231, grad_fn=<NllLossBac

tensor(0.0153, grad_fn=<NllLossBackward>)
tensor(0.0150, grad_fn=<NllLossBackward>)
tensor(0.0273, grad_fn=<NllLossBackward>)
tensor(0.0178, grad_fn=<NllLossBackward>)
tensor(0.0136, grad_fn=<NllLossBackward>)
tensor(0.0209, grad_fn=<NllLossBackward>)
tensor(0.0269, grad_fn=<NllLossBackward>)
tensor(0.0153, grad_fn=<NllLossBackward>)
tensor(0.0082, grad_fn=<NllLossBackward>)
tensor(0.0125, grad_fn=<NllLossBackward>)
tensor(0.0149, grad_fn=<NllLossBackward>)
tensor(0.0105, grad_fn=<NllLossBackward>)
tensor(0.0129, grad_fn=<NllLossBackward>)
tensor(0.8169, grad_fn=<NllLossBackward>)
tensor(0.0113, grad_fn=<NllLossBackward>)
tensor(0.0131, grad_fn=<NllLossBackward>)
tensor(0.0111, grad_fn=<NllLossBackward>)
tensor(0.0155, grad_fn=<NllLossBackward>)
tensor(0.0097, grad_fn=<NllLossBackward>)
tensor(0.3437, grad_fn=<NllLossBackward>)
tensor(0.0117, grad_fn=<NllLossBackward>)
tensor(0.0077, grad_fn=<NllLossBackward>)
tensor(0.0100, grad_fn=<NllLossBackward>)
tensor(0.4000, grad_fn=<NllLossBac

KeyboardInterrupt: 

huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
huhu
Eval loss: tensor(0.0723)
